# Cobre 

In [1]:
import os
import pandas as pd
from datetime import date, datetime
from dateutil.relativedelta import relativedelta

In [2]:
#Setando var bases
caminho = r"C:\Users\A0161597\OneDrive - Telefonica\Documentos\Rentabilidade\Relatorios\2025\04-ABRIL - Copia\REPOS"

caminhopadrao = r"C:\Users\A0161597\OneDrive - Telefonica\Documentos\Rentabilidade\Relatorios\2025\01-JANEIRO\JANEIRO25.xlsx"

meses = {
    1 : 'Jan',
    2 : 'Fev',
    3 : 'Mar',
    4 : 'Abr',
    5 : 'Mai',
    6 : 'Jun',
    7 : 'Jul',
    8 : 'Ago',
    9 : 'Set',
    10 : 'Out',
    11 : 'Nov',
    12 : 'Dez',
}

data = date.today()
ano_peq = date.today().strftime("%y")
data = pd.to_datetime("2025-04-01")

ultimos_12_meses = []

for i in range(0,12):
    ref = data - relativedelta(months=i)
    ref2 = f'{meses[int(ref.strftime("%m"))]}_{ref.strftime("%y")}_Val'
    ultimos_12_meses.append(ref2)

premissa_voz = 4.95
premissa_bl = 10.43

In [3]:
#Carregando arquivos

padrao_file = pd.ExcelFile(caminhopadrao)
padrao = padrao_file.sheet_names

energia_file = pd.ExcelFile(os.path.join(caminho,'ENERGIA.xlsx'))
energia = energia_file.sheet_names

aluguel_df = pd.read_csv(os.path.join(caminho,"ALUGUEL.csv"), sep='|')

receita_df = pd.read_csv(os.path.join(caminho,'RECEITA.txt'), sep='\t', decimal=',',encoding='latin-1')

hades_file = pd.ExcelFile(os.path.join(caminho,'HADES.xlsx'))
hades = hades_file.sheet_names

print(f'\nPadrao:\n{padrao}')
print(f'\nEnergia:\n{energia}')
print(f'\nHades:\n{hades}')

padrao_armario = padrao_file.sheet_names[0]
padrao_central = padrao_file.sheet_names[1]
energia = energia_file.sheet_names[0]
hades = hades_file.sheet_names[2]

padrao_armario_df = padrao_file.parse(padrao_armario)
padrao_central_df = padrao_file.parse(padrao_central)
energia_df = energia_file.parse(energia, header=3)
hades_df = hades_file.parse(hades, header=1)



Padrao:
['ARMARIO', 'CENTRAL', 'INFO HADES', 'RECEITA', 'ENERGIA', 'ALUGUEL', 'BUCLE CENTRAL', 'REDE EXT CENTRAL', 'REDE EXT E ACESSO ARM', 'PREMISSA', 'DE PARA AREA ARM', 'DE PARA ENERGIA', 'DE PARA CENTRAL', 'DE PARA ARMARIO']

Energia:
['Concessionaria']

Hades:
['Resumo', 'Vivo 1', 'Vivo 2', 'MSAN  - (VIVO1)', 'MINI_DSLAM', 'Planilha1', 'DESLIGUE']


In [4]:
#Tratamento Planilha Padrao
#parse todas as planilhas

planilhas_padrao = {
    'BUCLE CENTRAL',
    'REDE EXT CENTRAL',
    'REDE EXT E ACESSO ARM',
    'PREMISSA',
    'DE PARA AREA ARM',
    'DE PARA ENERGIA',
    'DE PARA CENTRAL',
    'DE PARA ARMARIO'
}

#ingestao todas as planilhas
for i in planilhas_padrao:
    var_name = f"df_{i.replace(' ', '_')}"
    print(var_name)
    globals()[var_name] = padrao_file.parse(i)


df_DE_PARA_ENERGIA = df_DE_PARA_ENERGIA[['COD', 'UF/ED']] # type: ignore
df_REDE_EXT_CENTRAL = df_REDE_EXT_CENTRAL.iloc[:, 0:11] # type: ignore
df_DE_PARA_ARMARIO = df_DE_PARA_ARMARIO.iloc[:, 2:5] # type: ignore
df_DE_PARA_AREA_ARM = df_DE_PARA_AREA_ARM[['NOME RENT.', 'AREA']] # type: ignore

df_DE_PARA_AREA_ARM.rename(columns={
    'NOME RENT.':'ARMARIO'
}, inplace=True)

df_REDE_EXT_E_ACESSO_ARM.rename(columns={ # type: ignore
    'MANUT TOTAL':'REDE EXT. + MANUT'
}, inplace=True)


df_REDE_EXT_E_ACESSO_ARM
df_DE_PARA_CENTRAL
df_DE_PARA_ENERGIA
df_DE_PARA_ARMARIO
df_REDE_EXT_CENTRAL
df_DE_PARA_AREA_ARM
df_BUCLE_CENTRAL
df_PREMISSA


In [5]:
#Trata planilha ENERGIA para join 

colunas = ['UFED']
colunas = colunas + ultimos_12_meses
energia_df = energia_df[colunas]

energia_df = energia_df.groupby('UFED')[ultimos_12_meses].sum().reset_index()

energia_df['REF'] = energia_df[ultimos_12_meses].mean(axis=1)

energia_df = pd.merge(energia_df, df_DE_PARA_ENERGIA, how='left', left_on='UFED', right_on='UF/ED')

energia_df = energia_df[['COD', 'REF']]

In [6]:
#Trata planilha aluguel para join 

colunas = ['pca_gp_r011_zrea.bnwhr','pca_gp_r011_zrea.recntxt','pca_gp_r011_zrea.dbervon_s']
aluguel_df = aluguel_df[colunas]

aluguel_df['pca_gp_r011_zrea.recntxt'] = aluguel_df['pca_gp_r011_zrea.recntxt'].str.replace(' ', '')

aluguel_df.rename(columns={
    'pca_gp_r011_zrea.bnwhr':'VALOR',
    'pca_gp_r011_zrea.recntxt':'CONTRATO',
    'pca_gp_r011_zrea.dbervon_s':'PERIODO'
}, inplace=True)

aluguel_df['PERIODO'] = pd.to_datetime(aluguel_df['PERIODO'], format='%Y%m')

aluguel_df = aluguel_df[(aluguel_df['PERIODO'] <= data) & (aluguel_df['PERIODO'] >= (data - relativedelta(months=12)))]

aluguel_df = aluguel_df.groupby('CONTRATO')['VALOR'].sum().div(12).reset_index()

aluguel_df = pd.merge(aluguel_df, df_DE_PARA_ARMARIO, how='left', left_on='CONTRATO', right_on='DE_PARA_ALUGUEL')

aluguel_df = aluguel_df[['ARMARIO', 'VALOR']]

In [7]:
#Trata planilha receita para join

receita_df = pd.read_csv(os.path.join(caminho,'RECEITA.txt'), sep='\t', decimal=',',encoding='latin-1')
receita_df = receita_df[receita_df['TECNOLOGIA_PRINICIPAL'].str.contains('2 - COBRE', na=False)]
receita_df = receita_df[['ARMARIO_PRINCIPAL','SEGMENTO','Sum(FAT_VOZ)','Sum(FAT_DADOS)', 'Sum(VLR_TOTAL)', 'Sum(QTD_VOZ_MULTI)', 'Sum(QTD_BL_MULTI)']]
receita_df.rename(columns={
    'ARMARIO_PRINCIPAL':'COD',
    'SEGMENTO':'SEG',
    'Sum(FAT_VOZ)':'FVOZ',
    'Sum(FAT_DADOS)':'FDADOS',
    'Sum(VLR_TOTAL)':'FTOTAL',
    'Sum(QTD_VOZ_MULTI)':'QTDVOZ',
    'Sum(QTD_BL_MULTI)':'QTDBL',
}, inplace=True)

colunas_str_to_float = ['FDADOS', 'FVOZ', 'FTOTAL']

for i in colunas_str_to_float:
    receita_df[i] = receita_df[i].astype(str).str.replace('.','').str.replace(',','.')
    receita_df[i] = pd.to_numeric(receita_df[i], errors='coerce')

receita_b2b = receita_df[receita_df['SEG'].str.contains('B2B', na=False)].copy()
receita_b2b.drop(columns='SEG', inplace=True)
receita_b2b = receita_b2b.groupby('COD').sum().reset_index()

receita_b2c = receita_df[receita_df['SEG'].str.contains('B2C', na=False)].copy()
receita_b2c.drop(columns='SEG', inplace=True)
receita_b2c = receita_b2c.groupby('COD').sum().reset_index()

receita_b2b.rename(columns={
    'FVOZ':'B2BFVOZ',
    'FDADOS':'B2BFDADOS',
    'FTOTAL':'B2BFTOTAL',
    'QTDVOZ':'B2BQTDVOZ',
    'QTDBL':'B2BQTDBL',
}, inplace=True)

receita_b2c.rename(columns={
    'FVOZ':'B2CFVOZ',
    'FDADOS':'B2CFDADOS',
    'FTOTAL':'B2CFTOTAL',
    'QTDVOZ':'B2CQTDVOZ',
    'QTDBL':'B2CQTDBL',
},  inplace=True)

receita_df = pd.merge(receita_b2b, receita_b2c, how='left', on='COD')

In [8]:
#Trata planilha hades para join

hades_df = hades_df[['Armários', 'Status']]
hades_df.rename(columns={
    'Armários': 'ARMARIO',
    'Status':'HADES'
}, inplace=True)

In [9]:
armario_df = padrao_armario_df[['TIPO', 'ARMARIO', 'COD_IBGE', 'DDD', 'REGIONAL','UF', 'CIDADE']]

armario_df = pd.merge(armario_df, df_DE_PARA_AREA_ARM, how='left', on='ARMARIO',suffixes=('', '_AREA'))
armario_df = pd.merge(armario_df, hades_df, how='left', on='ARMARIO',suffixes=('', '_HADES'))

receita_treated = receita_df[['COD', 'B2BFTOTAL','B2CFTOTAL']].copy()
receita_treated['RECEITA BRUTA'] = receita_treated[['B2BFTOTAL','B2CFTOTAL']].sum(axis=1).copy()
receita_treated['RECEITA LIQUIDA'] = receita_treated[['RECEITA BRUTA']] * 0.72
receita_treated['REC. RAT.'] = ''

ugr_treated = receita_df[['COD','B2BQTDVOZ', 'B2BQTDBL', 'B2CQTDVOZ', 'B2CQTDBL']].copy()
ugr_treated['UGR BL'] = ugr_treated[['B2BQTDBL', 'B2CQTDBL']].sum(axis=1).copy()
ugr_treated['UGR VOZ'] = ugr_treated[['B2BQTDVOZ', 'B2CQTDVOZ']].sum(axis=1).copy()

armario_df = pd.merge(armario_df, ugr_treated, how='left', left_on='ARMARIO', right_on='COD')

armario_df = pd.merge(armario_df, receita_treated, how='left', on='COD')

cond_hades = (armario_df['HADES'] != 'Desligado').astype(int)
cond_tipo = (armario_df['TIPO'] != 'SP').astype(int)

armario_df = pd.merge(armario_df, df_REDE_EXT_E_ACESSO_ARM[['AREA', 'REDE EXT. + MANUT']], how='left', on='AREA') # type: ignore

armario_df['REDE EXT. + MANUT'] = cond_hades * cond_tipo * armario_df['REDE EXT. + MANUT'].fillna(0)

armario_df['SERVIR'] = (armario_df['UGR BL'] * premissa_bl) + (armario_df['UGR VOZ'] * premissa_voz)

armario_df = pd.merge(armario_df, aluguel_df, how='left', on='ARMARIO')

armario_df = pd.merge(armario_df, energia_df, how='left', on='COD')

armario_df.rename(columns={
    'B2BQTDVOZ':'UGR VOZ B2B',
    'B2BQTDBL':'UGR DADOS B2B',
    'B2CQTDVOZ':'UGR VOZ B2C',
    'B2CQTDBL':'UGR DADOS B2C',
    'B2BFTOTAL':'RECEITA BRUTA B2B',
    'B2CFTOTAL':'RECEITA BRUTA B2C',
    'VALOR':'ALUGUEL',
    'REF':'ENERGIA',
}, inplace=True)

pd.set_option('display.max_columns', None)
armario_df.head()



,TIPO,ARMARIO,COD_IBGE,DDD,REGIONAL,UF,CIDADE,AREA,HADES,COD,UGR VOZ B2B,UGR DADOS B2B,UGR VOZ B2C,UGR DADOS B2C,UGR BL,UGR VOZ,RECEITA BRUTA B2B,RECEITA BRUTA B2C,RECEITA BRUTA,RECEITA LIQUIDA,REC. RAT.,REDE EXT. + MANUT,SERVIR,ALUGUEL,ENERGIA
0,FSP,MGBHE_O1A26,3106200,31,MG,MG,BELO HORIZONTE,0,Desligado,MGBHE_O1A26,3.0,2.0,NaN,NaN,2.0,3.0,49078.77,NaN,49078.77,35336.7144,,0.0000,35.71,NaN,NaN
1,FSP,RSPAE_O1A87,4314902,51,SUL,RS,PORTO ALEGRE,0,Desligado,RSPAE_O1A87,1.0,0.0,NaN,NaN,0.0,1.0,31675.37,NaN,31675.37,22806.2664,,0.0000,4.95,NaN,0.000000
2,FSP,RSPAE_O1I20,4314902,51,SUL,RS,PORTO ALEGRE,0,Desligado,RSPAE_O1I20,3.0,1.0,127.0,8.0,9.0,130.0,0.00,0.00,0.00,0.0000,,0.0000,737.37,NaN,491.805834
3,FSP,GOVPIOO1A02,5221858,61,CO,GO,VALPARAISO DE GOIAS,0,Desligado,GOVPIOO1A02,49.0,24.0,36.0,25.0,49.0,85.0,4081.30,4323.58,8404.88,6051.5136,,0.0000,931.82,2668.529167,18.961667
4,FSP,ALMCO_O1A01,2704302,82,NORDESTE,AL,MACEIO,AL-01,Bloqueado,ALMCO_O1A01,9.0,6.0,NaN,NaN,6.0,9.0,1603.60,NaN,1603.60,1154.5920,,1021.6955,107.13,1772.006667,675.644165


In [11]:
armario_df = armario_df[armario_df['ARMARIO'] == 'GOVPIOO1A02']
armario_df.head()

,TIPO,ARMARIO,COD_IBGE,DDD,REGIONAL,UF,CIDADE,AREA,HADES,COD,UGR VOZ B2B,UGR DADOS B2B,UGR VOZ B2C,UGR DADOS B2C,UGR BL,UGR VOZ,RECEITA BRUTA B2B,RECEITA BRUTA B2C,RECEITA BRUTA,RECEITA LIQUIDA,REC. RAT.,REDE EXT. + MANUT,SERVIR,ALUGUEL,ENERGIA
3,FSP,GOVPIOO1A02,5221858,61,CO,GO,VALPARAISO DE GOIAS,0,Desligado,GOVPIOO1A02,49.0,24.0,36.0,25.0,49.0,85.0,4081.3,4323.58,8404.88,6051.5136,,0.0,931.82,2668.529167,18.961667
